In [1]:
%cd D:\GitHub_Repository\Structural-Estimation\bisieco\notebooks\

D:\GitHub_Repository\Structural-Estimation\bisieco\notebooks


c:\Users\Tsunc\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd
import numpy as np

from scipy.special import digamma

import warnings
warnings.simplefilter('ignore')

In [3]:
theta_true = {'theta_c': 0.004, 'theta_p': 0.003}
beta = 0.99
euler_const = digamma(1)
num_choice = 2

states_p = list(range(2000,2600,100))
states_m = list(range(0,105,5))

num_states_p = len(states_p)
num_states_m = len(states_m)
num_states = num_states_p * num_states_m

state_df = pd.DataFrame({
    'state_id': np.arange(1, num_states + 1),
    'price_id': np.tile(np.arange(1, num_states_p + 1), num_states_m),
    'mileage_id': np.repeat(np.arange(1, num_states_m + 1), num_states_p),
    'price': np.tile(states_p, num_states_m),
    'mileage': np.repeat(states_m, num_states_p)
})

state_df


,state_id,price_id,mileage_id,price,mileage
0,1,1,1,2000,0
1,2,2,1,2100,0
2,3,3,1,2200,0
3,4,4,1,2300,0
4,5,5,1,2400,0
...,...,...,...,...,...
121,122,2,21,2100,100
122,123,3,21,2200,100
123,124,4,21,2300,100
124,125,5,21,2400,100


In [19]:
def gen_mileage_trans(kappa, num_states_m):
    """
    Generate the mileage transition matrix given parameters.

    Parameters
    ----------
    kappa : list or array-like
        [kappa_1, kappa_2], probabilities of moving up 1 or 2 mileage states.
    num_m_states : int
        Number of mileage states.

    Returns
    -------
    numpy.ndarray
        3D array with shape (num_mileage_states, num_mileage_states, num_choice),
        containing transition matrices for "not buy" and "buy".
    """
    # milage can increase in 1 or 2 step. And it's dependent on kappa_1 and kappa_2
    kappa_1, kappa_2 = kappa
        
    # not buy new car
    m_trans_not_buy = np.zeros((num_states_m, num_states_m))
    for i in range(num_states_m):
        for j in range(num_states_m):
            if i < j:
                m_trans_not_buy[i,j] = 0
            elif i==j:
                m_trans_not_buy[i,j] = 1 - kappa_1 - kappa_2
            elif i == j-1:
                m_trans_not_buy[i,j] = kappa_1
            elif i == j-2:
                m_trans_not_buy[i,j] = kappa_2
    m_trans_not_buy[num_states_m - 2, num_states_m - 1] = kappa_1 + kappa_2
    m_trans_not_buy[num_states_m - 1, num_states_m - 1] = 1
    
    # buy new car -> 
    m_trans_buy = np.ones((num_states_m, 1)) @ m_trans_not_buy[0, :].reshape(1, -1)
    
    mileage_trans = np.stack([m_trans_not_buy, m_trans_buy], axis=2)
    
    return mileage_trans

In [ ]:
def gen_price_trans(lmbda, num_states_p):
    """
    Generate the price transition matrix given lambda parameters.

    Parameters
    ----------
    lmbda : list or array-like
        Vector of 30 parameters (lambda[1] ~ lambda[30]).
    num_price_states : int
        Number of price states (default 6).

    Returns
    -------
    numpy.ndarray
        num_price_states x num_price_states price transition matrix.
    """

    lmbda = np.array(lmbda)

    # Compute diagonal terms
    lambda_11 = 1 - np.sum(lmbda[0:5])
    lambda_22 = 1 - np.sum(lmbda[5:10])
    lambda_33 = 1 - np.sum(lmbda[10:15])
    lambda_44 = 1 - np.sum(lmbda[15:20])
    lambda_55 = 1 - np.sum(lmbda[20:25])
    lambda_66 = 1 - np.sum(lmbda[25:30])

    # Construct full matrix row by row (same as in R)
    elements = [
        lambda_11, lmbda[0], lmbda[1], lmbda[2], lmbda[3], lmbda[4],
        lmbda[5], lambda_22, lmbda[6], lmbda[7], lmbda[8], lmbda[9],
        lmbda[10], lmbda[11], lambda_33, lmbda[12], lmbda[13], lmbda[14],
        lmbda[15], lmbda[16], lmbda[17], lambda_44, lmbda[18], lmbda[19],
        lmbda[20], lmbda[21], lmbda[22], lmbda[23], lambda_55, lmbda[24],
        lmbda[25], lmbda[26], lmbda[27], lmbda[28], lmbda[29], lambda_66
    ]

    # Convert to 6x6 matrix (by row)
    price_trans = np.array(elements).reshape(num_states_p, num_states_p, order='C')
    return price_trans

In [ ]:
kappa_true = [0.25, 0.05]
m_trans_true = gen_mileage_trans(kappa=kappa_true, num_states_m=num_states_m)

lmbda_true = [0.1, 0.2, 0.2, 0.2, 0.2,
              0.1, 0.2, 0.2, 0.2, 0.2,
              0.1, 0.1, 0.2, 0.2, 0.1,
              0.1, 0.1, 0.2, 0.2, 0.1,
              0.05, 0.05, 0.1, 0.1, 0.2,
              0.05, 0.05, 0.1, 0.1, 0.2]
p_trans_true = gen_price_trans(lmbda=lmbda_true, num_states_p=num_states_p)

trans_true = {}
trans_true['not_buy'] = np.kron(m_trans_true[:,:,0], p_trans_true)
trans_true['buy'] = np.kron(m_trans_true[:,:,1], p_trans_true)

In [36]:
# price distribution at steady state
# i.e. price_dist @ p_trans = price_dist

eigenvalues, eigenvectors = np.linalg.eig(p_trans_true.T)
idx = np.argmin(np.abs(eigenvalues - 1))
steady_state = np.real(eigenvectors[:, idx])

steady_state = steady_state / np.sum(steady_state)

steady_state

array([0.07377049, 0.07377049, 0.16393443, 0.16393443, 0.28571429,
       0.23887588])

In [ ]:
def flow_utility(theta, state):
    theta_c, theta_p = theta
    
    